In [28]:
import pandas as pd
import networkx as nx
from haversine import haversine, Unit
import itertools

In [29]:

NODE_FILE = 'road_network_nodes.csv'
EDGE_FILE = 'road_network_edges.csv'

# --- 2. 加载数据 ---
print("正在加载CSV文件...")
try:
    nodes_df = pd.read_csv(NODE_FILE)
    edges_df = pd.read_csv(EDGE_FILE)
    print("文件加载成功。")
    # 确保列名正确，如果你的列名不同，请在这里修改
    # 例如: nodes_df.rename(columns={'id': 'node_id', 'longitude': 'lon', 'latitude': 'lat'}, inplace=True)
except FileNotFoundError as e:
    print(f"错误: {e}")
    print("请确保CSV文件与脚本在同一目录下，或者提供完整路径。")
    exit()
    
nodes_df

正在加载CSV文件...
文件加载成功。


/var/folders/6y/7xz8vbkd0xdc45zvwsjfvwr80000gn/T/ipykernel_73929/3216938400.py:9: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv(EDGE_FILE)


,osmid,y,x,street_count,highway,ref,railway,geometry
0,150549126,35.897730,120.169790,2,NaN,NaN,NaN,POINT (120.1697901 35.8977301)
1,150583831,35.913610,120.178789,2,NaN,NaN,NaN,POINT (120.1787889 35.9136099)
2,150702730,35.902944,120.167157,2,NaN,NaN,NaN,POINT (120.1671568 35.9029436)
3,150881339,35.911360,120.176100,2,NaN,NaN,NaN,POINT (120.1761 35.91136)
4,150935729,35.895930,120.169700,2,NaN,NaN,NaN,POINT (120.1697001 35.89593)
...,...,...,...,...,...,...,...,...
64239,12973193890,36.001533,120.282159,3,NaN,NaN,NaN,POINT (120.2821591 36.001533)
64240,12973193891,36.001626,120.283112,3,NaN,NaN,NaN,POINT (120.283112 36.0016262)
64241,12973193892,36.001643,120.282841,2,NaN,NaN,NaN,POINT (120.2828408 36.0016433)
64242,12973193893,36.001590,120.282474,2,NaN,NaN,NaN,POINT (120.2824744 36.0015899)


In [30]:
# --- 3. 构建路网图 ---
print("正在构建路网图...")
G = nx.Graph()

# 添加节点，并存储经纬度信息
for _, row in nodes_df.iterrows():
    G.add_node(row['osmid'], lon=row['x'], lat=row['y'])

# 添加边，并设置权重
# **重要**: networkx的最短路径算法默认使用 'weight' 属性
# 如果你的 'length' 列不叫 'length'，请修改下面的 'length'
if 'length' in edges_df.columns:
    print("使用CSV中提供的 'length' 作为权重。")
    for _, row in edges_df.iterrows():
        G.add_edge(row['u'], row['v'], weight=row['length'])
else:
    # 如果edges.csv中没有length列，我们可以自己计算
    print("警告: 'length' 列未在edges.csv中找到。")
    print("正在根据节点坐标计算边的权重（直线距离）。")
    node_coords = {row['node_id']: (row['lat'], row['lon']) for _, row in nodes_df.iterrows()}
    for _, row in edges_df.iterrows():
        try:
            coord1 = node_coords[row['u']]
            coord2 = node_coords[row['v']]
            # 计算两点间的哈弗赛因距离（单位：米）
            distance = haversine(coord1, coord2, unit=Unit.METERS)
            G.add_edge(row['u'], row['v'], weight=distance)
        except KeyError as e:
            print(f"警告: 节点 {e} 在edges文件中存在，但在nodes文件中未找到。该边将被忽略。")


print(f"图构建完成。节点数: {G.number_of_nodes()}, 边数: {G.number_of_edges()}")

正在构建路网图...
使用CSV中提供的 'length' 作为权重。
图构建完成。节点数: 64244, 边数: 74043


In [31]:
# --- 4. 寻找最近的节点 ---
def find_nearest_node(graph, lat, lon):
    """
    在图中找到距离给定经纬度最近的节点
    """
    min_dist = float('inf')
    nearest_node = None
    for node, data in graph.nodes(data=True):
        dist = haversine((lat, lon), (data['lat'], data['lon']), unit=Unit.METERS)
        if dist < min_dist:
            min_dist = dist
            nearest_node = node
    return nearest_node


In [32]:
# 你的两个经纬度坐标 (纬度, 经度)
start_coord = (35.985924, 120.146173)  
end_coord = (35.979291, 120.161095)

OUTPUT_CSV_FILE = 'shortest_paths_coordinates.csv'
print("\n正在为起始和结束坐标寻找最近的路网节点...")
start_node = find_nearest_node(G, start_coord[0], start_coord[1])
end_node = find_nearest_node(G, end_coord[0], end_coord[1])

print(f"起始坐标 {start_coord} -> 最近节点: {start_node}")
print(f"结束坐标 {end_coord} -> 最近节点: {end_node}")

# --- 5. 计算4条最短路径 ---
if start_node is None or end_node is None:
    print("错误：无法找到有效的起始或结束节点。")
elif start_node == end_node:
    print("错误：起始节点和结束节点相同。")
else:
    print(f"\n正在计算从节点 {start_node} 到 {end_node} 的4条最短路径...")
    try:
        paths_generator = nx.shortest_simple_paths(G, source=start_node, target=end_node, weight='weight')
        k_shortest_paths = list(itertools.islice(paths_generator, 4))
        
        # --- 6. 显示结果并准备保存数据 ---
        if not k_shortest_paths:
            print("在路网中找不到任何从起点到终点的路径。")
        else:
            # 用于存储所有路径坐标的列表
            all_paths_data = []

            for i, path in enumerate(k_shortest_paths):
                total_length = nx.path_weight(G, path, weight='weight')
                
                print(f"\n--- 路径 {i+1} ---")
                print(f"  总长度: {total_length:.2f} 米")
                print(f"  节点数量: {len(path)}")
                print(f"  节点序列: {path}")

                # =======================================================
                # 新增部分：为当前路径的每个节点提取坐标并添加到列表中
                # =======================================================
                for j, node_id in enumerate(path):
                    # 从nodes_df中查找经纬度
                    node_coords = nodes_df.loc[nodes_df['osmid'] == node_id]
                    all_paths_data.append({
                        'path_id': i + 1,        # 路径编号 (1, 2, 3, ...)
                        'node_order': j,         # 节点在路径中的顺序 (0, 1, 2, ...)
                        'node_id': node_id,      # 节点ID
                        'lon': node_coords['x'].iloc[0],
                        'lat': node_coords['y'].iloc[0]
                    })

            # =======================================================
            # 新增部分：将所有路径数据转换为DataFrame并保存为CSV
            # =======================================================
            if all_paths_data:
                results_df = pd.DataFrame(all_paths_data)
                results_df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
                print(f"\n所有路径的坐标已成功保存到文件: '{OUTPUT_CSV_FILE}'")

            if len(k_shortest_paths) < 4:
                print(f"\n注意: 在路网中只找到了 {len(k_shortest_paths)} 条路径，少于您要求的4条。")

    except nx.NetworkXNoPath:
        print(f"在路网中找不到任何从节点 {start_node} 到 {end_node} 的路径。")


正在为起始和结束坐标寻找最近的路网节点...
起始坐标 (35.985924, 120.146173) -> 最近节点: 7772768010
结束坐标 (35.979291, 120.161095) -> 最近节点: 9177336621

正在计算从节点 7772768010 到 9177336621 的4条最短路径...

--- 路径 1 ---
  总长度: 1802.26 米
  节点数量: 17
  节点序列: [7772768010, 3685215587, 7388326849, 3685215588, 9177336635, 3685215582, 3265488916, 3265488941, 2914709816, 3265488947, 3265488974, 5431842916, 5431842914, 5329452974, 3685215516, 5329452969, 9177336621]

--- 路径 2 ---
  总长度: 1807.97 米
  节点数量: 16
  节点序列: [7772768010, 11074527802, 9177336637, 9177336635, 3685215582, 3265488916, 3265488941, 2914709816, 3265488947, 3265488974, 5431842916, 5431842914, 5329452974, 3685215516, 5329452969, 9177336621]

--- 路径 3 ---
  总长度: 1813.83 米
  节点数量: 19
  节点序列: [7772768010, 3685215587, 7388326849, 3685215588, 2914708398, 5431843476, 5431843479, 3265488980, 3265488914, 2914709884, 3265488986, 3265488971, 2914720202, 5431842915, 5431842914, 5329452974, 3685215516, 5329452969, 9177336621]

--- 路径 4 ---
  总长度: 1813.88 米
  节点数量: 19
  节点序列: [77727